# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,25.86,76,5,7.50,PF,1675538226
1,1,vaini,-21.2000,-175.2000,27.09,89,20,4.63,TO,1675538228
2,2,huilong,31.8111,121.6550,3.29,80,100,2.76,CN,1675538228
3,3,hithadhoo,-0.6000,73.0833,27.38,73,100,7.23,MV,1675538229
4,4,cidreira,-30.1811,-50.2056,25.02,70,86,5.22,BR,1675538229


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",frame_width = 800,frame_height = 600,scale = 0.5,color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
#max temp  bewteen 21 and 27, wind speed < 4.5, cloudiness = 0
filter_city_data_df = city_data_df[(city_data_df["Max Temp"] >21)&(city_data_df["Max Temp"] <27)&(city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"]==0)]


# Drop any rows with null values
filter_city_data_df.dropna()

# Display sample data
filter_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,bredasdorp,-34.5322,20.0403,22.00,83,0,0.32,ZA,1675538231
58,58,devarkonda,16.7000,78.9333,22.64,42,0,1.62,IN,1675538253
89,89,port elizabeth,-33.9180,25.5701,24.06,90,0,2.06,ZA,1675538265
205,205,sur,22.5667,59.5289,21.42,70,0,1.43,OM,1675538307
221,221,otane,-39.8833,176.6333,21.12,90,0,0.37,NZ,1675538315
403,403,richards bay,-28.7830,32.0377,22.52,88,0,1.16,ZA,1675538383
503,503,jati,24.3539,68.2671,21.05,66,0,3.61,PK,1675538421


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filter_city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,bredasdorp,ZA,-34.5322,20.0403,83,
58,devarkonda,IN,16.7000,78.9333,42,
89,port elizabeth,ZA,-33.9180,25.5701,90,
205,sur,OM,22.5667,59.5289,70,
221,otane,NZ,-39.8833,176.6333,90,
403,richards bay,ZA,-28.7830,32.0377,88,
503,jati,PK,24.3539,68.2671,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
devarkonda - nearest hotel: udipi hotel
port elizabeth - nearest hotel: Waterford Hotel
sur - nearest hotel: Sur Hotel
otane - nearest hotel: Te Aute Hotel
richards bay - nearest hotel: Protea Hotel
jati - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,bredasdorp,ZA,-34.5322,20.0403,83,Victoria Hotel
58,devarkonda,IN,16.7000,78.9333,42,udipi hotel
89,port elizabeth,ZA,-33.9180,25.5701,90,Waterford Hotel
205,sur,OM,22.5667,59.5289,70,Sur Hotel
221,otane,NZ,-39.8833,176.6333,90,Te Aute Hotel
403,richards bay,ZA,-28.7830,32.0377,88,Protea Hotel
503,jati,PK,24.3539,68.2671,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",frame_width = 800,frame_height = 600,scale = 0.5,color = "City",hover_cols = ["Country", "Hotel Name"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)